In [ ]:
# This magic makes plots appear 
%matplotlib inline
import matplotlib.pyplot as plt
import pyvista as pv

import os
os.environ["GADOPT_LOGLEVEL"] = "WARNING"

# Load Firedrake on Colab
try:
    import firedrake
except ImportError:
    !wget "https://github.com/g-adopt/tutorials/releases/latest/download/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

# 2-D Cylindrical Case
## Weak formulation
For our reference we take another look at our weak formulation:<br>

$$\int_\Omega (\nabla \vec{v})\colon \mu \left[ \nabla \vec{u} + \left( \nabla \vec{u} \right)^T\right] \ dx 
 - \int_{\Omega} \left( \nabla \cdot \vec{v}\right)\ p \ dx
 - \int_{\Omega} Ra_0\ T\ \vec{v}\cdot\hat{k} \ dx = 0 \ \text{ for all } v\in V,$$

$$ \int_\Omega w \nabla \cdot \vec{u} \ dx\ \text{ for all } v\in V,$$

$$  \int_\Omega q\frac{\partial T}{\partial t} \ dx
  + \int_\Omega q \vec{u}\cdot \nabla T \ dx 
  + \int_\Omega \left(\nabla q\right) \cdot \left(\kappa \nabla T\right) \ dx = 0   \text{ for all } q\in Q.$$


In the Cartesian examples we saw earlier are imposed through strong Dirichlet boundary conditions for velocity $\vec{u}$. This is achieved by restricting the velocity function space $V$ to a subspace $V_0$
of vector functions for which all components (zero-slip) or only the normal
component (free-slip) are zero at the boundary. Since this restriction also
applies to the test functions $\vec{v}$, the weak form only needs to be
satisfied for all test functions $\vec{v}\in V_0$ that satisfy the homogeneous boundary conditions. Therefore, the omitted boundary integral
\begin{equation}
  -\int_{\partial\Omega} \vec{v}\cdot \left(\mu \left[\nabla\vec{u}
    + \left(\nabla\vec{u}\right)^T\right]\right)\cdot \vec{n} ds
\end{equation}
that was required to obtain the integrated by parts viscosity term, automatically vanishes for zero-slip boundary conditions as
$\bf v =0$ at the domain boundary, $\partial\Omega$. In the case of a free-slip
boundary condition for which the tangential components of $\vec{v}$ are
non-zero, the boundary term does not vanish, but by omitting that term, we weakly impose a zero shear stress condition. The boundary
term obtained by integrating the pressure gradient term by parts,
\begin{equation}
  \int_{\partial\Omega} \vec{v}\cdot\vec{n} p ds ,
\end{equation}


Note that we have integrated by parts the viscosity and pressure gradient terms in the Stokes equations, and the diffusion term in the energy equation, but have omitted the corresponding boundary terms. Our goal in this section is to examine simulations in a 2-D cylindrical domain.<br> 

We define our domain by the radii of the inner ($r_{\text{min}}$) and outer ($r_{\text{max}}$) boundaries. These are chosen such that the non--dimensional depth of the mantle,
$z = r_{\text{max}} - r_{\text{min}} = 1$, and the ratio of the inner and outer radii, $f=r_{\text{min}} / r_{\text{max}} = 0.55$, thus approximating the ratio between the
radii of Earth's surface and core-mantle-boundary (CMB). Specifically, we set $r_{\text{min}} = 1.22$ and $r_{\text{max}} = 2.22$.<br>




In [ ]:
from gadopt import *
import numpy as np

In [ ]:
# Set up geometry:
rmin, rmax, ncells, nlayers = 1.22, 2.22, 32, 8

mesh1d = CircleManifoldMesh(ncells, radius=rmin, degree=2)
mesh = ExtrudedMesh(mesh1d, layers=nlayers, extrusion_type='radial')

bottom_id, top_id = "bottom", "top"
n = FacetNormal(mesh)
domain_volume = assemble(1 * dx(domain=mesh))

### Exercise 7.1:
- Visualise the generated mesh.

- Change radial and lateral resolution and visualise the mesh again. 


In [ ]:
V = FunctionSpace(mesh, "CG", 1)
File("mesh.pvd").write(Function(V))
mesh_data = pv.read("mesh_0.vtu")
edges = mesh_data.extract_all_edges()

plotter = pv.Plotter()
plotter.add_mesh(edges, color="black")
plotter.camera_position = "xy"

fig = plt.figure(figsize=(10, 10))
plt.imshow(plotter.show(interactive=False, return_img=True))
plt.axis("off");

We now need the function spaces that are associated with this mesh. We follow the same definitions as our last exercise, and use the following definitions: 


In [ ]:
# Set up function spaces - currently using the bilinear Q2Q1 element pair:
V = VectorFunctionSpace(mesh, "CG", 2)  # Velocity function space (vector)
W = FunctionSpace(mesh, "CG", 1)  # Pressure function space (scalar)
Q = FunctionSpace(mesh, "CG", 2)  # Temperature function space (scalar)
Z = MixedFunctionSpace([V, W])  # Mixed function space.

z = Function(Z)  # a field over the mixed function space Z.
u, p = split(z)  # Returns symbolic UFL expression for u and p


We choose the initial temperature distribution in a way so that our convection run produces 4 equidistant plumes. This initial temperature field is prescribed as:
$$T(x,y) = (r_{\text{max}} - r) + A\cos(4 \; atan2\ (y,x))  \sin(r-r_{\text{min}}) \pi)$$
where $A=0.02$ is the amplitude of the initial perturbation. 

In [ ]:
# Set up temperature field and initialise:
X = SpatialCoordinate(mesh)
r = sqrt(X[0]**2 + X[1]**2)
T = Function(Q, name="Temperature")
T.interpolate(rmax - r + 0.02*cos(4*atan_2(X[1], X[0])) * sin((r - rmin) * pi))

In [ ]:
# Let's visualise the initial temperature field now
File("temp.pvd").write(T)
temp_data = pv.read("temp_0.vtu")
plotter = pv.Plotter()
plotter.add_mesh(temp_data)
plotter.camera_position = "xy"

fig = plt.figure(figsize=(10, 10))
plt.imshow(plotter.show(interactive=False, return_img=True))
plt.axis("off");

Next, we strategise our solutions for the energy equations. We use $\theta$ time-stepping scheme for our enery solutions, and define our way of computing a time step consistent with a cfl condition number of 1.0.

In [ ]:
delta_t = Constant(1e-7)  # Initial time step
t_adapt = TimestepAdaptor(delta_t, V, maximum_timestep=0.1, increase_tolerance=1.5)

# Define time stepping parameters:
steady_state_tolerance = 1e-7
max_timesteps = 2000 
time = 0.0

## Boundary Condition: Introducting Weak Imposition of Boundary Conditions
Boundary conditions for temperature are $T = 0$ at the surface ($r_{\text{max}}$) and $T = 1$ at the base ($r_{\text{min}}$).<br>
For velocity boundary conditions we choose free‐slip velocity boundary conditions that are specified on both boundaries. We incorporate these <b>weakly</b> through the <i>Nitsche</i> approximation.
### Weak Imposition of Boudary Conditions - Nitsche Penalty Method
In curved domains, such as the 2-D cylindrical and 3-D spherical cases examined below, imposing free-slip boundary conditions 
is complicated by the fact that it is not straightforward to decompose the degrees of freedom of the velocity space V
into tangential and lateral components for many finite element discretisations. For Lagrangian based discretisations we could
define normal vectors at the Lagrangian nodes on the surface and decompose accordingly, but these normal vectors would have
to be averaged due to the piecewise approximation of the curved surface. To avoid such complications for our examples in
cylindrical and spherical geometries, we employ <i>a symmetric Nitsche penalty method</i>  where the velocity space
is not restricted and, thus, retains all discrete solutions with a non-zero normal component.<br>

This entails adding the following three surface integrals:
\begin{equation}
  - \int_{\partial\Omega} \vec{v}\cdot\vec{n}~
    \vec{n}\cdot \left(\mu \left[\nabla\vec{u}
    + \left(\nabla\vec{u}\right)^T\right]\right)\cdot \vec{n} \ ds
  - \int_{\partial\Omega} 
    \vec{n}\cdot \left(\mu \left[\nabla\vec{v}
    + \left(\nabla\vec{v}\right)^T\right]\right)\cdot \vec{n}~
    \vec{u}\cdot\vec{n} \ ds
  + \int_{\partial\Omega} C_{\text{Nitsche}} \, \mu \vec{v}\cdot\vec{n} ~ \vec
  u\cdot\vec{n} \ ds .
\end{equation}


The first of these corresponds to the normal component of stress 
associated with integration by parts of the viscosity term. The tangential
component, as before, is omitted and weakly imposes a zero shear stress
condition. The second term ensures symmetry of Equation \eqref{eq:weak_mom} 
with respect to $\vec{u}$ and $\vec{v}$. The third term penalizes the normal
component of $\vec{u}$ and involves a penalty parameter $C_{\text{Nitsche}}>0$
that should be sufficiently large to ensure convergence.

<br>
coercivity of the bilinear form $F_{\text{Stokes}}$ introduced in
Lower bounds for $C_{\text{Nitsche},f}$ on each face $f$ can be derived for simplicial \citep{shahbazi_2005} and quadrilateral/hexahedral \citep{hillewaert_2013} meshes

Finally for the energy equation, we apply a simple $\theta$ scheme 

In [ ]:
# Stokes related constants
Ra = Constant(1e5)  # Rayleigh number
approximation = BoussinesqApproximation(Ra)

temp_bcs = {
    bottom_id: {"T": 1.0},
    top_id: {"T": 0.0},
}

stokes_bcs = {
    bottom_id: {"un": 0},
    top_id: {"un": 0},
}

### Nullspaces and near-nullspaces 
With a free-slip boundary condition on both boundaries, one can add an arbitrary rotation of the form $(-y, x)=r\hat{\mathbf{\theta}}$ to the velocity solution (i.e. this case incorporates a velocity nullspace, as well as a pressure nullspace). These lead to null-modes (eigenvectors) for the linear system, rendering the resulting matrix singular. In preconditioned Krylov methods these null-modes must be subtracted from the approximate solution at every iteration.

By using the GAMG preconditioner, we must provide near-nullspace information for the GAMG preconditioner, consisting of three rotational (`x_rotV`, `y_rotV`, `z_rotV`) and three translational (`nns_x`, `nns_y`, `nns_z`) modes (see GMD paper for details). 

We now update the Stokes problem to account for additional boundary conditions, and the Stokes solver to include the near nullspace options defined above, in addition to the optional `appctx` keyword argument that passes the viscosity through to our `MassInvPC` Schur complement preconditioner. Energy solver options are also updated relative to our base case, using the dictionary that is created before.


In [ ]:
Z_nullspace = create_stokes_nullspace(Z, closed=True, rotational=True)
Z_near_nullspace = create_stokes_nullspace(Z, closed=False, rotational=True, translations=[0, 1])

energy_solver = EnergySolver(T, u, approximation, delta_t, ImplicitMidpoint, bcs=temp_bcs)
stokes_solver = StokesSolver(z, T, approximation, bcs=stokes_bcs,
                             cartesian=False,
                             nullspace=Z_nullspace, transpose_nullspace=Z_nullspace,
                             near_nullspace=Z_near_nullspace)

### Main time stepping
We can now initiate the time-loop, with the Stokes system solved seperately. These `solve` calls once again convert symbolic mathematics into computation. In the time loop, set here to run until we reach a steady-state (see `maxchange`), we compute the RMS velocity and surface Nusselt number for diagnostic purposes, and print these results every 50 timesteps. 

In [ ]:
for timestep in range(0, max_timesteps):
    dt = t_adapt.update_timestep(u)
    time += dt

    # Solve Stokes sytem:
    stokes_solver.solve()

    # Temperature system:
    energy_solver.solve()

    # Compute diagnostics:
    u_rms = sqrt(assemble(dot(u, u) * dx)) * sqrt(1./domain_volume)
    f_ratio = rmin/rmax
    top_scaling = -1.3290170684486309  # log(f_ratio) / (1.- f_ratio)
    bot_scaling = -0.7303607313096079  # (f_ratio * log(f_ratio)) / (1.- f_ratio)
    nusselt_number_top = (assemble(dot(grad(T), n) * ds_t) / assemble(1 * ds_t(domain=mesh))) * top_scaling
    nusselt_number_base = (assemble(dot(grad(T), n) * ds_b) / assemble(1 * ds_b(domain=mesh))) * bot_scaling
    energy_conservation = abs(abs(nusselt_number_top) - abs(nusselt_number_base))
    average_temperature = assemble(T * dx) / domain_volume

    # Calculate L2-norm of change in temperature:
    maxchange = sqrt(assemble((T - energy_solver.T_old)**2 * dx))

    if timestep % 100 == 0 or maxchange < steady_state_tolerance:
        print(f"{u_rms=}, Nu_t={nusselt_number_top}, Nu_b={nusselt_number_base}, {maxchange=}")

    # Leave if steady-state has been achieved:
    if maxchange < steady_state_tolerance:
        log("Steady-state achieved -- exiting time-step loop")
        break

In [ ]:
# Let's visualise the initial temperature field now
File("temp.pvd").write(T)
temp_data = pv.read("temp_0.vtu")
plotter = pv.Plotter()
plotter.add_mesh(temp_data)
plotter.camera_position = "xy"

fig = plt.figure(figsize=(10, 10))
plt.imshow(plotter.show(interactive=False, return_img=True))
plt.axis("off");

### Exercise 7.2
- Change the top boundary conditions from free-slip to no-slip (zero velocity) in the following script. The appropriate boundary condition can be set by using `DirichletBC`. Notice the change in `nullspaces` for this problem. 